In [1]:
from selenium import webdriver
from time import sleep
import pandas as pd
import csv
import urllib, requests
import numpy as np
from nltk import flatten
import dask
from dask import delayed,compute
import dask.dataframe as dd
from dask.multiprocessing import get
from tqdm._tqdm_notebook import tqdm_notebook
import seaborn as sns
import matplotlib.pyplot as plt
from tqdm import tqdm
tqdm_notebook.pandas()
import time

<ipython-input-1-2c35a00995ff>:12: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.*` instead of `tqdm._tqdm_notebook.*`
  from tqdm._tqdm_notebook import tqdm_notebook
C:\Users\Dell\anaconda3\lib\site-packages\tqdm\std.py:668: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [2]:
import spacy, collections
import en_core_web_trf 
import warnings
warnings.filterwarnings('ignore')

In [3]:
import re
import tensorflow as tf
import tensorflow_hub as hub
import nltk
from selenium.webdriver.common.keys import Keys

In [25]:
file_name = "python_libraries - python_libraries.csv"
with open(file_name,newline="") as f:
    reader = csv.reader(f)
    data = list(reader)

In [6]:
arr = np.array(data) 
print(arr.shape)

(215, 4)


In [7]:
driver = webdriver.Firefox()

In [8]:
def clean_text(sentence):
    if sentence:
        tokens = [word.lemma_ for word in nlp(str(sentence)) 
                  if not (word.like_num 
                          or word.is_stop
                          or word.is_punct
                          or word.is_space
                          or word.like_url
                          or word.like_email
                          or word.is_currency
                          or word.pos_ =='VBZ' 
                          or word.pos_ =='ADP'
                          or word.pos_ =='PRON'
                          or word.pos_ =='AUX'

                         )] 
    else :
        return np.nan
    return tokens

In [9]:
def extract_abbreviations(text):
    abbr_fullform_dict=dict()
    abbr = re.findall('\(([A-Z]{2,})\)',text)
    #print(abbr)
    if len(abbr)<10:
        for i in abbr:
            span_abbr = re.search(i,text).span()
            len_full_form = str(span_abbr[1]-span_abbr[0])
            try:
                full_form = re.search('(\w+\s)'+'{'+len_full_form+'}'+'\('+i+'\)',text).group(0)
                if full_form[0].lower()!= i.lower()[0]:
                    full_form = ' '.join(full_form.split()[1:])
                if full_form.lower()[-1]!= i.lower()[-1]:
                    full_form = ' '.join(full_form.split()[:-1])
                full_form = full_form.replace('('+i+')','').strip()
                abbr_fullform_dict[i] = full_form
            except AttributeError:
                print("error")
                pass
    
        return abbr_fullform_dict

In [10]:
def remove_abbriviation(text,abbr_dict):
    try :
       
        #abbr_dict = ast.literal_eval(abbr_dict)
    
        for key,value in abbr_dict.items():
                #print(key,value)
                text = text.replace(key,value)
    except AttributeError:            
          pass
    return text

In [11]:
def remove_email(text):
    if text:
        text = text.lower()
        text = re.sub('([a-zA-Z0-9_\-\.]+)@([a-zA-Z0-9_\-\.]+)\.([a-zA-Z]{2,5})',' ',str(text))
    else:
        pass
    return text

In [12]:
def remove_weblink(text):
    if text:
        text=text.lower()
        text = re.sub('(https?:\/\/(?:www\.|(?!www))[a-zA-Z0-9][a-zA-Z0-9-]+[a-zA-Z0-9]\.[^\s]{2,}|www\.[a-zA-Z0-9][a-zA-Z0-9-]+[a-zA-Z0-9]\.[^\s]{2,}|https?:\/\/(?:www\.|(?!www))[a-zA-Z0-9]+\.[^\s]{2,}|www\.[a-zA-Z0-9]+\.[^\s]{2,})','link',str(text))
    else:
        pass
    return text

In [13]:
def remove_refrence(text):
    if text:
        text=text.lower()
        text = re.sub('\[\d+(,\s{0,}\d+){0,}\]','',str(text))   
    else:
        pass
    return text

In [14]:
# Function for removing non ASCII charecters like 'Ϫ','ó','ü','©','µ','▲','→'
# This function check if charecter hex value in range [\x00,\x7F] (in decimal [0,127] i.e range of ASCII charecters) and replace if it occurs outside limit
def remove_ghost_char(text):
    if text:
        text = re.sub(r'[^\x00-\x7F]+',' ', str(text))
    else:
        pass
    
    return text

In [15]:
# This function remove all bracktes with data
def remove_brackets(text):
    if text:
        text = re.sub('(\(.*?\))|(\[.*?\])','',str(text))   
    else:
        pass
    return text

In [16]:
# Function for removing multiple spaces 
def remove_extra_spaces(text):
    if text:
        text = re.sub(r'( +)',' ', str(text))
    else: 
        pass
   
    return text

In [17]:
def preprocess(url_text,dask_=False ,remove_abbr_=False,remove_email_=True,remove_weblink_=True,
               remove_refrence_=True,remove_brackets_=True,remove_ghost_char_=True,remove_extra_spaces_=True):
    start_time = time.time()
                          
    if remove_email_: 
        for i in range(len(url_text)):
            s = ""
            for j in range(len(url_text[i])):
                s+=url_text[i][j]+"\n"
            url_text[i] = remove_email(s).split("\n")
            
    if remove_weblink_: 
        for i in range(len(url_text)):
            s = ""
            for j in range(len(url_text[i])):
                s+=url_text[i][j]+"\n"
            url_text[i] = remove_weblink(s).split("\n")
            
    if remove_refrence_: 
        print('Removing refrences now!')
        for i in range(len(url_text)):
            s = ""
            for j in range(len(url_text[i])):
                s+=url_text[i][j]+"\n"
            url_text[i] = remove_refrence(s).split("\n")                   
                          
    if remove_brackets_: 
        print('Removing brackets now!')
        for i in range(len(url_text)):
            s = ""
            for j in range(len(url_text[i])):
                s+=url_text[i][j]+"\n"
            url_text[i] = remove_brackets(s).split("\n")                
    if remove_ghost_char_: 
        print('Removing bad charecters now!')
        for i in range(len(url_text)):
            s = ""
            for j in range(len(url_text[i])):
                s+=url_text[i][j]+"\n"
            url_text[i] = remove_ghost_char(s).split("\n")                 
    if remove_extra_spaces_:
        print('Removing Extra spaces now!')
        for i in range(len(url_text)):
            s = ""
            for j in range(len(url_text[i])):
                s+=url_text[i][j]+"\n"
            url_text[i] = remove_extra_spaces(s).split("\n")
#         if dask_:                       
#             _series =(dd
#                                .from_pandas(_series, npartitions=4)
#                                .apply(lambda x: remove_extra_spaces(x),meta=('text_body', 'object'))
#                                .compute(scheduler='processes')
#                                 )
#         else:
#              _series = _series.apply(lambda x: remove_extra_spaces(x))                 
    print ("completed preprocessing text in {:2f} minutes".format((time.time()-start_time)/60))
    
    return url_text

In [18]:
#Calculate cosine similarity of query with all sentences
def cosine_similarity_func(embeddings,embeddings_query):
    '''
    Input:
         embeddings: array or tensor of all sentence embeddings (nX128 for n sentences)
         embeddings_query: array or tensor of query embedding (1X128)
    Output:
         cosine_similarity: cosine similarity of query with each sentence (nX1) 
    '''
    # x.y
    dot_product = np.sum(np.multiply(np.array(embeddings),np.array(embeddings_query)),axis=1)
    
    #||x||.||y||
    prod_sqrt_magnitude = np.multiply(np.sum(np.array(embeddings)**2,axis=1)**0.5, np.sum(np.array(embeddings_query)**2,axis=1)**0.5)
    
    #x.y/(||x||.||y||)
    cosine_similarity  = dot_product/prod_sqrt_magnitude
    return cosine_similarity

In [19]:
# function for recommend text based upon query
def recommended_text(query,embeddings,sent,threshold_min=.95,threshold_max = 1):
    '''
    Input:
         query: list of queries
         embeddings: embeddings of all sentences
         sent:list all sentences
         threshold_min: lower limit of threshold for which sentence is supposed to be similar with query
         threshold_max: upper limit of threshold for which sentence is supposed to be similar with query
         
    Output:
          recommend_text: list of similar sentences with query
    '''
    recommend_text = []
    embeddings_query = embed(query) #create embedding for query
    
    cosine_similarity = cosine_similarity_func(embeddings,embeddings_query) # get cosine similarity with all sentences
    
    # standardize cosine similarity output, Range(0,1)
    standardize_cosine_simi  = (cosine_similarity-min(cosine_similarity))/(max(cosine_similarity)-min(cosine_similarity))
    
    #sort sent based upon cosine similarity score
    sent_prob = list(map(lambda x, y:(x,y), standardize_cosine_simi, sent)) 
    sent_prob.sort(key=lambda tup: tup[0], reverse=True)

    # select sentences by using upper and lower threshold
    for i,j in sent_prob:
        if (i >threshold_min) and (i<=threshold_max):
            recommend_text.append(j)
    return recommend_text

In [20]:
file = open('deprecation_information2.csv', 'w+', encoding="utf-8", newline ='')
write = csv.writer(file)

In [34]:
module_url = "https://tfhub.dev/google/nnlm-en-dim128/2"
embed = hub.KerasLayer(module_url)

In [32]:
import nltk

In [35]:
for i in range(len(data)):
    d = data[i]
    url = d[1]
    git_url = d[2]
    xpath = d[3]
    print(d)
    if(url == "-" or url==""):
        continue
    try:
        driver.get(url)
    except:
        print(url)
    name = driver.title
    hrefs = []
    if(xpath!="-" and xpath!=''):
        try:
            a_tags = driver.find_elements_by_xpath(xpath)
        except:
            continue
        for a in a_tags:
            hrefs.append(a.get_attribute('href'))
    if(len(hrefs)==0):
        hrefs.append(url)
    url_text = []
    for i in range(len(hrefs)):
        try:
            driver.get(hrefs[i])
        except:
            continue
#         driver.send_keys(Keys.CONTROL + Keys.HOME)
#         driver.send_keys(Keys.CONTROL + Keys.END)
        elem = driver.find_element_by_tag_name('body')
#         continue_link = driver.find_element_by_tag_name('a')
#         in_page_links = driver.find_elements_by_xpath("//*[@href]")
        elem_text = []
#         for link in in_page_links:
#             driver.get(link)
#             elem = driver.find_element_by_tag_name('body')
        elem_text = elem_text + elem.text.split("\n")
        url_text.append(elem_text)
    nlp = en_core_web_trf.load()
#     nlp.max_length = 2000000
    url_text = preprocess(url_text)
    seen = nltk.flatten(url_text)
    embeddings = embed(seen)
    embeddings.shape
    query = ["is deprecated and replaced with"]
    try:
        result = recommended_text(query,embeddings,seen,threshold_min=.6)
    except:
        continue
    data1 = [name]+result
    print(data1[:5])
    write.writerow(data1)

['https://github.com/psf/requests', 'https://docs.python-requests.org/en/latest/community/updates/#release-history', 'https://github.com/psf/requests/releases', '-']
Removing refrences now!
Removing brackets now!
Removing bad charecters now!
Removing Extra spaces now!
completed preprocessing text in 0.002039 minutes
['Community Updates — Requests 2.25.1 documentation', 'the requests extra is officially deprecated and will be removed in requests v2.26.0.', 'updated included ca bundle with new mistrusts and automated process for the future', 'features and improvements', 'features and improvements']
['https://github.com/pandas-dev/pandas', 'https://pandas.pydata.org/docs/whatsnew/index.html', 'https://github.com/pandas-dev/pandas/releases', '//*[(@id = "bd-docs-nav")]//a']
Removing refrences now!
Removing brackets now!
Removing bad charecters now!
Removing Extra spaces now!
completed preprocessing text in 0.016981 minutes
['Release notes — pandas 1.2.4 documentation', 'the parameter numer

Removing refrences now!
Removing brackets now!
Removing bad charecters now!
Removing Extra spaces now!
completed preprocessing text in 0.001042 minutes
['Release notes - Albumentations Documentation', 'albumentations now explicitly checks that all inputs to augmentations are named arguments and raise an exception otherwise. so if an augmentation receives input like aug instead of aug, albumentations will raise an exception. ', 'api for a.fda is changed to resemble api of a.histogrammatching. now, both transformations expect to receive a list of reference images, a function to read those image, and additional augmentation parameters. ', 'fixed a bug that led to an exception when a.load was called to deserialize a pipeline that contained a.totensor or a.totensorv2, but those transforms were not imported in the code before the call. ', 'getting started with albumentations']
['https://github.com/OpenMined/PySyft', '-', 'https://github.com/OpenMined/PySyft/releases', '-']
['https://github.c

Removing refrences now!
Removing brackets now!
Removing bad charecters now!
Removing Extra spaces now!
completed preprocessing text in 0.000000 minutes
['Release History - Stanza', '1.0.1 2020-04-27 this is a maintenance release of stanza. it features new support for jieba as chinese tokenizer, faster lemmatizer implementation, improved compatibility with corenlp v4.0.0, and several bugfixes including correct character offsets in ner output and correct vietnamese tokenization outputs .', '0.2.0 2019-05-16 this release introduces substantially reduced model size, substantial lemmatizer speed up and more options for customizing server start up and requests .', '1.1.1 2020-08-13 this release features support for extending the capability of the stanza pipeline with customized processors, a new sentiment analysis tool for english/german/chinese, improvements to the corenlpclient functionality , new models for a few languages , new biomedical and clinical english packages, alternative server

Removing refrences now!
Removing brackets now!
Removing bad charecters now!
Removing Extra spaces now!
completed preprocessing text in 0.005909 minutes
['Release history — Trio 0.18.0+dev documentation', "trio.queue's join and task_done methods are deprecated without replacement ", 'remove all the apis deprecated in 0.9.0 or earlier , trio.brokenstreamerror, and trio.resourcebusyerror), except for trio.hazmat.unboundedqueue, which stays for now since it is used by the obscure lowlevel functions monitor_completion_queue and monitor_kevent. ', 'run_sync_in_worker_thread is deprecated in favor of trio.to_thread.run_sync. ', 'the nursery context manager was rewritten to avoid use of @asynccontextmanager and @async_generator. this reduces extraneous frames in exception traces and addresses bugs regarding stopiteration and stopasynciteration exceptions not propagating correctly. ']
['https://github.com/PyCQA/isort', 'https://pycqa.github.io/isort/CHANGELOG/', 'https://github.com/PyCQA/isort/

['Changelog - LSP Mode - LSP support for Emacs', 'lsp-define-stdio-client and lsp-define-tcp-client are replaced with lsp-register-client', 'imenu support is now handled asynchronously .', 'breaking change: bindings for commands under s were moved under w for better compatibility with spacemacs', 'performed several fixes to make sure lsp-mode is working fine with emacs 28 native compilation.']
['https://github.com/pygame/pygame', 'https://www.pygame.org/whatsnew.shtml', 'https://github.com/pygame/pygame/releases', '']
Removing refrences now!
Removing brackets now!
Removing bad charecters now!
Removing Extra spaces now!
completed preprocessing text in 0.010325 minutes
["What's New", ' config_msys.py and config.py os.popen is replaced with the newer', ' try again with os x and base_test.py.', ' close issue #237, memory overrun when text without descenders is rendered with underlining', ' lines with width>2 are now correct ']
['https://github.com/connorferster/handcalcs', '', 'https://git

Removing refrences now!
Removing brackets now!
Removing bad charecters now!
Removing Extra spaces now!
completed preprocessing text in 0.002694 minutes
['pyserial/CHANGES.rst at master · pyserial/pyserial · GitHub', 'improved xreadlines, it is now a generator function that yields lines as they are received . however xreadlines is deprecated and not available when the io module is used. use for line in serial: instead.', 'win32: setrts and setdtr can be called before the port is opened and it will set the initial state on port open.', 'miniterm.py has a new feature to send a file and configurable special characters for exit and upload. refactored internals to class based structure ', 'poll based implementation for read is in a separate class posixpollserial, as it is not supported well on all platforms .']
['https://github.com/home-assistant-libs/pychromecast', 'https://pyup.io/changelogs/pychromecast/', 'https://github.com/home-assistant-libs/pychromecast/releases', '']
Removing refren

Removing refrences now!
Removing brackets now!
Removing bad charecters now!
Removing Extra spaces now!
completed preprocessing text in 0.000508 minutes
['instagram_private_api/CHANGELOG.md at master · ping/instagram_private_api · GitHub', 'client.standard_ratios and client.reel_ratios is deprecated and replaced with mediaratios.standard and mediaratios.reel respectively.', 'deprecated and experimental endpoints now warned with clientdeprecationwarning and clientexperimentalwarning.', 'can be used to largely replace the public instagram api that is now severely restricted', 'user_feed is now using the graphql endpoint and has compatibility-breaking changes.']
['https://github.com/nayuki/QR-Code-generator', '', 'https://github.com/nayuki/QR-Code-generator/releases', '']
['https://github.com/jameslyons/python_speech_features', '', 'https://github.com/jameslyons/python_speech_features/releases', '']
['https://github.com/RJT1990/pyflux', '', 'https://github.com/RJT1990/pyflux/releases', '']

['scrapely/NEWS at master · scrapy/scrapely · GitHub', '- replaced deprecated functions', '- add c extension to speed up parsing and extraction', '- number extractor now supports numbers with a sign.', '- add python3 support for `url_to_page` function']
['https://github.com/rlabbe/filterpy', 'https://github.com/rlabbe/filterpy/blob/master/filterpy/changelog.txt', 'https://github.com/rlabbe/filterpy/releases', '']
Removing refrences now!
Removing brackets now!
Removing bad charecters now!
Removing Extra spaces now!
completed preprocessing text in 0.000179 minutes
['filterpy/changelog.txt at master · rlabbe/filterpy · GitHub', 'now the sigma point and weight generation is done with a separate class,', '* sped up computation in cubature and ensemble filters by using ', 'classes now have attributes wm and wc, and weights was deleted.', '* added better default values for f and h in kalmanfilter']
['https://github.com/pgmpy/pgmpy', 'https://github.com/pgmpy/pgmpy/blob/dev/CHANGELOG.md', 'htt

Removing refrences now!
Removing brackets now!
Removing bad charecters now!
Removing Extra spaces now!
completed preprocessing text in 0.000260 minutes
['shodan-python/CHANGELOG.md at master · achillean/shodan-python · GitHub', 'new api methods api.search_facets and api.search_filters to get a list of available facets and filters.', 'convert line endings of shodan/client.py and tests/test_shodan.py to unix', 'add history and type parameters to shodan.dns.domain_info method and cli command', 'include timestamp, vulns and tags in csv converter ']
['https://github.com/nerevu/riko', '', 'https://github.com/nerevu/riko/releases', '']
['https://github.com/dateutil/dateutil', 'https://dateutil.readthedocs.io/en/stable/changelog.html', 'https://github.com/dateutil/dateutil/releases', '']
Removing refrences now!
Removing brackets now!
Removing bad charecters now!
Removing Extra spaces now!
completed preprocessing text in 0.001687 minutes
['Changelog — dateutil 2.8.1 documentation', 'the use of 

Removing refrences now!
Removing brackets now!
Removing bad charecters now!
Removing Extra spaces now!
completed preprocessing text in 0.000168 minutes
['4. Release Notes — Buildbot 3.1.1 documentation', 'removed support for deprecated builder categories which have been replaced by tags.', 'avatar caching is now working properly and size argument is now handled correctly.', 'removed deprecated support for constructing build steps from class and arguments in buildfactory.addstep.', 'a new www.ws_ping_interval configuration option was added to avoid websocket timeouts when using reverse proxies and cdns ']
['https://github.com/volatilityfoundation/volatility', 'https://www.volatilityfoundation.org/releases', 'https://github.com/volatilityfoundation/volatility/releases', '//p[@class="font_8"]/span/a[@target="_self"]']
Removing refrences now!
Removing brackets now!
Removing bad charecters now!
Removing Extra spaces now!
completed preprocessing text in 0.000369 minutes
['Release Downloads |